In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
from tqdm import tqdm
import itertools
import os
import pandas as pd
import pickle
import random

np.random.seed(123)
torch.manual_seed(123)


## **Environment Class**

In [3]:
class BanditEnvironment:
    def __init__(self, max_steps = 100):
        self.num_actions = 11  # Number of actions (arms/bandits)
        self.max_steps = max_steps  # Maximum number of steps per episode
        self.reset()


    def reset(self):
        """Reset the environment to its initial state."""

        # Obtain the target bandit for the episode (the one with higer variance).
        self.target_bandit = np.random.randint(low = 0, high=10, size=None, dtype=int)
        # All bandits with Reward = 1
        self.bandits_reward = np.ones(11, dtype=np.float32)
        # A target bandit with reward = 5
        self.bandits_reward[self.target_bandit] = 5
        # A informative bandit that its reward is 1/10 the target bandit (considering bandits from 1 to 10, index from 0 to 9)
        self.bandits_reward[10] = np.round(np.float32((self.target_bandit + 1)/10),1)

        self.timestep = 0  # Initialize timestep
        self.last_action = None
        self.last_reward = 0
        return self.get_state()

    def step(self, action):
        """Take an action and return the next state, reward, and done flag."""
        self.timestep += 1
        self.last_action = action
        reward = self.bandits_reward[action] # Get reward
        self.last_reward = reward
        done = self.timestep >= self.max_steps  # End episode after 100 timesteps
        return self.get_state(), reward, done

    def get_state(self):
        """Return the current state: timestep, last action, and last reward."""

        # OH last action
        one_hot_action = np.zeros(self.num_actions)
        if self.last_action is not None:
          one_hot_action[self.last_action] = 1

        # OH reward
        one_hot_reward = np.zeros(11) # 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1, 5
        # Si se eligio el target, se marca oh recompensa = 5
        if self.last_action == self.target_bandit:
          one_hot_reward[10] = 1
        # Si se eligio otro no informativo, se marca oh recompensa = 1
        elif (self.last_action != 10) and self.last_action is not None:
          one_hot_reward[9] = 1
        # Si se elgio el informativo, el index del brazo target
        elif self.last_action == 10:
          one_hot_reward[self.target_bandit] = 1

        return np.concatenate([one_hot_action, one_hot_reward, np.array([self.timestep], dtype=np.float32)])


In [32]:
## Example Environement
env = BanditEnvironment()
state = env.reset()
print("Bandit Initial State (action, r, ts):", state)
print("Target Bandit (index):", env.target_bandit)
print("Bandits Reward:", env.bandits_reward)
action = 3
print(f"Taking action {action} (index)....")
next_state, reward, done = env.step(action)
print("Reward:", reward)
print("Next Bandit State:", next_state)
action = env.target_bandit
print(f"Taking action {action} (index)....")
next_state, reward, done = env.step(action)
print("Reward:", reward)
print("Next Bandit State:", next_state)
action = 10
print(f"Taking action {action} (index)....")
next_state, reward, done = env.step(action)
print("Reward:", reward)
print("Next Bandit State:", next_state)


Bandit Initial State (action, r, ts): [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
Target Bandit (index): 7
Bandits Reward: [1.  1.  1.  1.  1.  1.  1.  5.  1.  1.  0.8]
Taking action 3 (index)....
Reward: 1.0
Next Bandit State: [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1.]
Taking action 7 (index)....
Reward: 5.0
Next Bandit State: [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2.]
Taking action 10 (index)....
Reward: 0.8
Next Bandit State: [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 3.]


In [26]:
def metrics(total_reward):

    # Cumulative Regret
    optimal_reward = 5 * max_steps
    cumulative_regret = optimal_reward - total_reward

    return cumulative_regret

In [33]:
class A2C_LSTM_Agent(nn.Module):
    def __init__(self, input_dim, hidden_size, num_actions):
        super(A2C_LSTM_Agent, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_size, batch_first=True)
        self.actor_layer = nn.Linear(hidden_size, num_actions)
        self.critic_layer = nn.Linear(hidden_size, 1)
        nn.init.xavier_uniform_(self.actor_layer.weight, gain=0.01)
        nn.init.xavier_uniform_(self.critic_layer.weight, gain=1.0)

    def forward(self, x, hidden_state):
        # x shape: (batch_size, seq_len, input_dim)
        lstm_out, hidden_state = self.lstm(x, hidden_state)
        lstm_out = lstm_out[:, -1, :]  # Use the output of the last timestep
        action_probs = F.softmax(self.actor_layer(lstm_out), dim=-1)
        value = self.critic_layer(lstm_out)
        return action_probs, value, hidden_state

    def init_hidden_state(self, batch_size=1):
        # Initialize the LSTM hidden and cell states
        return (torch.zeros(1, batch_size, self.hidden_size).to(device),
                torch.zeros(1, batch_size, self.hidden_size).to(device))

In [34]:
def train_agent(TRAIN = True, n_episodes = 25000, gamma = 0.99, learning_rate = 0.001, entropy_decay = False):

    stats = {'Episode Returns': [], 'Optimal Action': [], 'Bandits_Reward': [], 'Cumulative Regret': [], 'Actions-Chosen List': [], 'First Action': []}

    for episode in tqdm(range(1, n_episodes + 1)):

        # Restart Constants, Environmentes and LSTM Hidden-State
        actions_chosen_list = [] if TRAIN == False else None
        actions_chosen = np.zeros(num_actions)
        state = env.reset()
        state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0).unsqueeze(0)  # Add batch and sequence dimensions
        hidden_state = agent.init_hidden_state()

        total_reward = 0
        done = False

        while not done:

            # Forward pass
            action_probs, value, hidden_state = agent(state, (hidden_state[0].detach(), hidden_state[1].detach()))

            # Sample action from probability distribution
            action_dist = torch.distributions.Categorical(action_probs)
            action = action_dist.sample()
            log_prob = action_dist.log_prob(action)

            # Take action in the environment
            next_state, reward, done = env.step(action.item())
            total_reward += reward

            # Prepare next state
            next_state = torch.FloatTensor(next_state).unsqueeze(0).unsqueeze(0).to(device)

            # Trainning
            if TRAIN == True:

                # Compute TD target and advantage
                with torch.no_grad():
                    _, next_value, _ = agent(next_state, hidden_state) #if not done else (None, torch.tensor(0.0), None)
                    td_target = reward + gamma * next_value
                advantage = td_target - value

                # Compute losses
                actor_loss = -log_prob * advantage.detach()
                critic_loss = F.mse_loss(value, td_target)
                entropy_loss = -action_dist.entropy().mean()
                entropy_coef = max(0.005, 0.1 * (1 - episode / n_episodes)) if entropy_decay == True else 0.005

                loss = actor_loss + 0.05 * critic_loss + entropy_coef * entropy_loss

                # Optimize the network
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            else:
                actions_chosen_list.append(action.item())

            # Move to the next state
            state = next_state

            # Add action
            actions_chosen[action.item()] += 1

            # Save First Action
            if env.timestep == 1:
              first_action = env.last_action

        # Cumulative Regret:
        cumulative_regret_value = metrics(total_reward)

        #Save Episode stats
        stats['Episode Returns'].append(total_reward)
        stats['Optimal Action'].append(env.target_bandit)
        stats['Bandits_Reward'].append(env.bandits_reward)
        stats['Actions-Chosen List'].append(actions_chosen_list)
        stats['Cumulative Regret'].append(cumulative_regret_value)
        stats['First Action'].append(first_action)

        if len(stats['Cumulative Regret']) >= 100:
            mean_last_100_regret = np.mean(stats['Cumulative Regret'][-100:]).round(1)
        else:
            mean_last_100_regret = np.mean(stats['Cumulative Regret']).round(1)

        if episode % 100 == 0:
            print(f"Ep {episode}/{n_episodes}, Opt. Action: {env.target_bandit}, Reward: {total_reward}, Cumulative-Regret: {cumulative_regret_value}, AVG100-Regret: {mean_last_100_regret}, First Action {first_action}")

    return stats



In [35]:
def save_train_stats(stats, model_name, discount_factor_gamma, learning_rate, entropy_decay):

    stats = pd.DataFrame(stats)

    stats['model_name'] = model_name
    stats['entropy_decay'] = entropy_decay
    stats.to_pickle(google_drive_folder + '/' + 'train_stats' + '/' + model_name + '.pkl')

def save_test_stats(stats, model_name, discount_factor_gamma, learning_rate, entropy_decay):

    stats = pd.DataFrame(stats)

    stats['model_name'] = model_name
    stats['entropy_decay'] = entropy_decay
    stats.to_pickle(google_drive_folder + '/' + 'test_stats' + '/' + model_name + '.pkl')



In [36]:
# Set Google Drive Folder
google_drive_folder = './Monografia/Exp3/'
os.makedirs(google_drive_folder + 'models', exist_ok=True)
os.makedirs(google_drive_folder + 'train_stats', exist_ok=True)
os.makedirs(google_drive_folder + 'test_stats', exist_ok=True)

# Hyperparameters
GAMMA_LIST = [0.8, 0.9, 1]
LR_LIST = [0.0001, 0.001]
ENTROPY_DECAY = [False, True]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hidden_size = 48
max_steps = 5
n_episodes_train = 25000
n_episodes_test= 300


hyperparameters = list(itertools.product(GAMMA_LIST, LR_LIST, ENTROPY_DECAY))
random.shuffle(hyperparameters)

for i, (gamma, learning_rate, entropy_decay) in enumerate(hyperparameters):

    print(f'\nGAMMA {gamma} - LR {learning_rate} - Entropy Decay {entropy_decay}')

    # Environment and agent setup
    env = BanditEnvironment(max_steps = max_steps)
    num_actions = env.num_actions
    input_dim = 1 + num_actions + 11  # Timestep, past action oh, past reward oh
    agent = A2C_LSTM_Agent(input_dim, hidden_size, num_actions).to(device)
    optimizer = optim.Adam(agent.parameters(), lr=learning_rate)

    # Train
    stats = train_agent(TRAIN = True, n_episodes = n_episodes_train, gamma = gamma, learning_rate = learning_rate, entropy_decay = entropy_decay)

    # Save Model and Train Stats
    agent.eval()
    model_name = f'LR_{str(learning_rate).replace(".","_")}__GAMMA_{str(gamma).replace(".","_")}__EntropyDecay_{str(entropy_decay)}'
    torch.save(agent.state_dict(), google_drive_folder + 'models/'+ model_name + '.pth')
    save_train_stats(stats, model_name, gamma, learning_rate, entropy_decay)

    print(f'\nTEST:')
    # Test and save stats
    stats = train_agent(TRAIN = False, n_episodes = n_episodes_test, entropy_decay = entropy_decay)
    save_test_stats(stats, model_name, gamma, learning_rate, entropy_decay)




GAMMA 0.9 - LR 0.0001 - Entropy Decay True


  0%|          | 104/25000 [00:02<11:57, 34.71it/s]

Ep 100/25000, Opt. Action: 2, Reward: 4.300000011920929, Cumulative-Regret: 20.69999998807907, AVG100-Regret: 18.0, First Action 8


  1%|          | 210/25000 [00:05<08:20, 49.50it/s]

Ep 200/25000, Opt. Action: 5, Reward: 4.600000023841858, Cumulative-Regret: 20.399999976158142, AVG100-Regret: 18.3, First Action 10


...

Ep 100/300, Opt. Action: 9, Reward: 5.0, Cumulative-Regret: 20.0, AVG100-Regret: 18.2, First Action 5


 76%|███████▌  | 228/300 [00:01<00:00, 138.21it/s]

Ep 200/300, Opt. Action: 3, Reward: 5.0, Cumulative-Regret: 20.0, AVG100-Regret: 17.3, First Action 4


100%|██████████| 300/300 [00:02<00:00, 138.02it/s]


Ep 300/300, Opt. Action: 6, Reward: 5.0, Cumulative-Regret: 20.0, AVG100-Regret: 18.0, First Action 0


## **Resultados**

In [48]:
# List Test Stats files

df_resultados = pd.DataFrame()
df_resultados['Action'] = range(0,11)

for file in os.listdir(google_drive_folder + 'test_stats'):

  if file.startswith('LR'):
    
    df = pd.read_pickle(google_drive_folder + 'test_stats/' + file)
    df = df['First Action'].value_counts().sort_index()
    file_column_name = file.split('.')[0]
    df = df.rename(file_column_name)
    df_resultados = df_resultados.merge(df, left_on = 'Action', right_index=True, how = 'left')

df_resultados = df_resultados.fillna(0).mean(axis=1)
(df_resultados / df_resultados.sum()*100).round(1)

0      7.3
1      2.9
2      9.3
3     10.5
4     24.3
5     11.5
6      2.9
7     21.8
8      6.4
9      2.7
10     0.3
dtype: float64

In [49]:
# List Test Stats files

df_resultados = pd.DataFrame()
df_resultados['Action'] = range(0,11)

for file in os.listdir(google_drive_folder + 'train_stats'):

  if file.startswith('LR'):
    
    df = pd.read_pickle(google_drive_folder + 'train_stats/' + file)
    df = df['First Action'].value_counts().sort_index()
    file_column_name = file.split('.')[0]
    df = df.rename(file_column_name)
    df_resultados = df_resultados.merge(df, left_on = 'Action', right_index=True, how = 'left')

df_resultados = df_resultados.fillna(0).mean(axis=1)
(df_resultados / df_resultados.sum()*100).round(1)

0      6.6
1      5.0
2     13.8
3     11.5
4     18.5
5      9.1
6      4.0
7     20.0
8      6.3
9      4.6
10     0.6
dtype: float64

## **Trying different seeds**

In [ ]:
SEEDS = [303, 55, 7, 101, 455]

# Set Google Drive Folder
google_drive_folder = '/content/drive/MyDrive/DataMining/Aprendizaje Reforzado/Monografia/Exp3/'
os.makedirs(google_drive_folder + 'models', exist_ok=True)
os.makedirs(google_drive_folder + 'train_stats', exist_ok=True)
os.makedirs(google_drive_folder + 'test_stats', exist_ok=True)

# Hyperparameters
GAMMA_LIST = [0.8, 0.9, 1]
LR_LIST = [0.0001, 0.001, 0.01, 0.1]
ENTROPY_DECAY = [False, True]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

hidden_size = 48
max_steps = 5
n_episodes_train = 25000
n_episodes_test= 300


hyperparameters = list(itertools.product(GAMMA_LIST, LR_LIST, ENTROPY_DECAY))
random.shuffle(hyperparameters)

for seed in SEEDS:
    print(f"SEED: {seed}")

    np.random.seed(seed)
    torch.manual_seed(seed)

    for i, (gamma, learning_rate, entropy_decay) in enumerate(hyperparameters):

        print(f'\nGAMMA {gamma} - LR {learning_rate} - Entropy Decay {entropy_decay}')

        # Environment and agent setup
        env = BanditEnvironment(max_steps = max_steps)
        num_actions = env.num_actions
        input_dim = 1 + num_actions + 1  # Timestep, past action oh, past reward
        agent = A2C_LSTM_Agent(input_dim, hidden_size, num_actions).to(device)
        optimizer = optim.Adam(agent.parameters(), lr=learning_rate)

        # Train
        stats = train_agent(TRAIN = True, n_episodes = n_episodes_train, gamma = gamma, learning_rate = learning_rate, entropy_decay = entropy_decay)

        # Save Model and Train Stats
        agent.eval()
        model_name = f'SEED_{str(seed)}__LR_{str(learning_rate).replace(".","_")}__GAMMA_{str(gamma).replace(".","_")}__EntropyDecay_{str(entropy_decay)}'
        torch.save(agent.state_dict(), google_drive_folder + 'models/'+ model_name + '.pth')
        save_train_stats(stats, model_name, gamma, learning_rate, entropy_decay)

        print(f'\nTEST:')
        # Test and save stats
        stats = train_agent(TRAIN = False, n_episodes = n_episodes_test, entropy_decay = entropy_decay)
        save_test_stats(stats, model_name, gamma, learning_rate, entropy_decay)



SEED: 303

GAMMA 1 - LR 0.01 - Entropy Decay True


  0%|          | 108/25000 [00:03<08:18, 49.97it/s]

Ep 100/25000, Opt. Action: 0, Reward: 25.0, Cumulative-Regret: 0.0, AVG100-Regret: 17.9, First Action 0


  1%|          | 208/25000 [00:05<08:21, 49.42it/s]

Ep 200/25000, Opt. Action: 9, Reward: 5.0, Cumulative-Regret: 20.0, AVG100-Regret: 17.6, First Action 0


...

 20%|█▉        | 4907/25000 [02:02<07:49, 42.83it/s]

Ep 4900/25000, Opt. Action: 0, Reward: 5.0, Cumulative-Regret: 20.0, AVG100-Regret: 18.8, First Action 8


 20%|██        | 5007/25000 [02:04<07:27, 44.67it/s]

Ep 5000/25000, Opt. Action: 5, Reward: 5.0, Cumulative-Regret: 20.0, AVG100-Regret: 17.6, First Action 8


 20%|██        | 5027/25000 [02:05<07:40, 43.33it/s]